# Document Rectifier

#### Import Test Images

### Get corners of document

### Find the homography to map the image to the new space

### Perform the warping operation

### Fully automated document recovery